# Differences that pre-processing cause to the baseline model in i2b2 data

for reference, command that was run within scripts/ was ```CUDA_VISIBLE_DEVICES=<device_no> python main.py --<cross_validate/use_test> --dataset=i2b2 --preprocessing_type=<entity_blinding/punct_digit/punct_stop_digit> --num_epoches=100 --lr_values 0.001 0.0001 --lr_boundaries 70```
This was gotten after preliminary hyperparameter tuning

In [23]:
from scipy.stats import ttest_rel
def paired_ttest(score1, score2):
    all_three_macroF1_score1 = [x for x in zip(*score1)]
    all_three_macroF1_score2 = [x for x in zip(*score2)]
    ttests = [ttest_rel(macro_f1_score1, macro_f1_score2) 
            for macro_f1_score1, macro_f1_score2 in zip(all_three_macroF1_score1, all_three_macroF1_score2)]
    print('8 way evaluation: \t', ttests[0])
    print('Problem-Treatment: \t', ttests[1])
    print('Problem-Test: \t\t', ttests[2])
    print('Problem-Problem: \t\t', ttests[3])

## First compare the cross validated score differences

In [2]:
baseline_test = (84.37, 68.76, 90.68, 90.6)
# model ID 6198ab41-3183-40f3-9254-d86a2b26e4ed on gray

In [15]:
# Still need to run this baseline
#baseline = currently running on harrison Feb 15, 2019
# temp baseline for now
# baseline = [(90.35, 84.26, 92.58, 92.86), (88.71, 77.25, 92.89, 93.27), (89.57, 81.2, 92.55, 93.16), 
#             (86.16, 75.21, 89.89, 91.82), (87.79, 78.66, 92.47, 89.47)]
baseline = [(89.65, 83.48, 91.88, 92.04), (88.47, 79.31, 91.69, 92.31), (90.52, 83.62, 92.59, 94.02), 
            (88.07, 78.79, 92.35, 90.35), (88.73, 81.67, 92.11, 90.52)]
# model ID de365f82-b85d-415a-acb5-c43d7e7f4040 on gray

In [16]:
entity_blinding = [(92.94, 86.21, 92.78, 100.0), (95.76, 91.77, 95.63, 100.0), 
                   (94.79, 87.83, 95.85, 100.0), (95.7, 92.92, 94.79, 100.0), (92.96, 83.4, 94.85, 99.56)]
# model ID 05668eeb-dc44-4fd1-8006-be1f65afc3c7 on gray

In [17]:
punct_digit = [(89.65, 84.98, 91.37, 91.48), (88.94, 80.0, 91.88, 93.21), (90.52, 83.9, 92.55, 93.97), 
               (89.5, 82.3, 91.91, 92.58), (89.2, 81.36, 92.95, 90.99)] 
# model ID 9121bf1e-557e-46c6-9a96-d2bb394179bd on gray

In [18]:
punct_stop_digit = [(90.12, 83.62, 90.82, 95.58), (86.32, 77.12, 90.36, 88.99), 
                    (90.76, 84.07, 92.43, 94.47), (85.2, 75.32, 88.71, 89.38), (88.03, 81.33, 90.96, 90.18)] 
# model ID 3da2c7cd-9ea0-4f79-aee4-0e714ee3146d on harrison

In [19]:
paired_ttest(baseline, entity_blinding)

8 way evaluation: 	 Ttest_relResult(statistic=-6.04357421784175, pvalue=0.00378085898207631)
Problem-Treatment: 	 Ttest_relResult(statistic=-2.7201162988132555, pvalue=0.052980501355252736)
Problem-Test: 		 Ttest_relResult(statistic=-5.233703287291305, pvalue=0.006367561601030022)
Problem-Problem: 		 Ttest_relResult(statistic=-12.780918540534149, pvalue=0.0002159650050905726)


In [20]:
paired_ttest(baseline, punct_digit)

8 way evaluation: 	 Ttest_relResult(statistic=-1.8154924056411164, pvalue=0.1436273789862555)
Problem-Treatment: 	 Ttest_relResult(statistic=-1.7104519892330639, pvalue=0.16235478342416396)
Problem-Test: 		 Ttest_relResult(statistic=-0.032705136678672427, pvalue=0.9754766119353693)
Problem-Problem: 		 Ttest_relResult(statistic=-1.2562526214204068, pvalue=0.2773904990319427)


In [21]:
paired_ttest(baseline, punct_stop_digit)

8 way evaluation: 	 Ttest_relResult(statistic=1.5273361018956584, pvalue=0.20139105244924393)
Problem-Treatment: 	 Ttest_relResult(statistic=1.4381306059681662, pvalue=0.22378129537918404)
Problem-Test: 		 Ttest_relResult(statistic=2.5334682029257674, pvalue=0.06442657659461247)
Problem-Problem: 		 Ttest_relResult(statistic=0.1151074482298427, pvalue=0.9139068913632511)


In [22]:
paired_ttest(punct_digit, punct_stop_digit)

8 way evaluation: 	 Ttest_relResult(statistic=1.644629753521851, pvalue=0.17539138703962795)
Problem-Treatment: 	 Ttest_relResult(statistic=1.6911369980810078, pvalue=0.16607134645237095)
Problem-Test: 		 Ttest_relResult(statistic=2.7096210431538323, pvalue=0.053558274460305755)
Problem-Problem: 		 Ttest_relResult(statistic=0.49438617773243926, pvalue=0.6469541658397595)


In short, entity blinding is greatly beneficial for the i2b2 data, because it probably helps the classifier in striations between the different relation types that are specific to the problem, test and treatment. Removing punctuations and normalizing digits has no effect, and removing stop words causes no difference except for Problem-Test relations. 

Test score results for the above are (all model IDs on gray):

```(84.37, 68.76, 90.68, 90.6)``` for baseline model with ID 6198ab41-3183-40f3-9254-d86a2b26e4ed

```(91.48, 80.05, 94.13, 100.0)``` for entity blinding model ID 0b4e4ece-fdd5-4ab8-8ecb-293cea44f0b6

```(84.77, 70.75, 90.29, 90.39)``` for punctuation removal and digit normalization with model ID 058d543a-447c-4c30-8328-6ff5312384af

```(82.54, 67.23, 89.07, 87.55)``` for punctuation and stop word removal and digit normalization with model ID dbf983d2-d6ef-4be4-a1fa-6270775ec93d

In the case of i2b2 datasets, information about the concepts is greatly beneficial in being able to classify the relations